# TP5 - Naive Bayes and & Logistic Regression : 
---
_Author: CHRISTOFOROU Anthony_\
_Due Date: XX-XX-2023_\
_Updated: 29-11-2023_\
_Description: TP4 - AI_

---

In [2]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

# Modules

# make figures appear inline
matplotlib.rcParams['figure.figsize'] = (15, 8)
%matplotlib inline

# notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2